In [1]:
!pip install numpy pandas matplotlib scikit-learn joblib torch

   ---------------------------------------- 0.0/113.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/113.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/113.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/113.8 MB ? eta -:--:--
   ---------------------------------------- 0.5/113.8 MB 693.3 kB/s eta 0:02:44
   ---------------------------------------- 0.5/113.8 MB 693.3 kB/s eta 0:02:44
   ---------------------------------------- 0.8/113.8 MB 705.7 kB/s eta 0:02:41
   ---------------------------------------- 1.0/113.8 MB 731.0 kB/s eta 0:02:35
   ---------------------------------------- 1.0/113.8 MB 731.0 kB/s eta 0:02:35
   ---------------------------------------- 1.3/113.8 MB 756.2 kB/s eta 0:02:29
    --------------------------------------- 1.6/113.8 MB 767.5 kB/s eta 0:02:27
    --------------------------------------- 1.6/113.8 MB 767.5 kB/s eta 0:02:27
    --------------------------------------- 1.8/113.8 MB 765.8 kB/s eta 0:02

In [45]:
#Step1  :  Imports & Settings

import os
import re
import json
import joblib
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import IsolationForest

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

warnings.filterwarnings("ignore")
np.set_printoptions(suppress=True)

In [46]:
#Step-2  :  Configurations

DATA_PATH = "../data/raw/fuel_dispenses.csv"          
OUTDIR = "ml_outputs"                     

STATION_ID = "STATION_01"
LOOKBACK = 14                             # past 14 days -> predict next day
EPOCHS = 30                               
BATCH_SIZE = 64
LR = 1e-3
SEED = 42

TRAIN_QUANTILE = 0.80                     
SCORE_QUANTILE = 0.95     

In [56]:
#Step-3  :  Utilities

def set_seeds(seed: int):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def to_float(x):
    """Convert '46,398.31' -> 46398.31 safely."""
    if pd.isna(x):
        return np.nan
    s = str(x).strip().replace(",", "")
    if s == "" or s.lower() == "nan":
        return np.nan
    try:
        return float(s)
    except Exception:
        return np.nan

def normalize_fuel(item: str) -> str:
    """Map report fuel names to stable codes."""
    s = str(item).lower()
    if "petrol" in s and "92" in s: return "PETROL_92"
    if "petrol" in s and "95" in s: return "PETROL_95"
    if "diesel" in s and ("auto" in s or "super" in s): return "DIESEL_AUTO"
    if "diesel" in s: return "DIESEL"
    return re.sub(r"[^a-z0-9]+", "_", s).strip("_").upper()

def compute_metrics(y_true, y_pred):
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)

    # Safe-MAPE: ignore tiny actuals (prevents explosion)
    eps = 50.0  # you can tune: 10, 25, 50 depending on your scale
    mask = np.abs(y_true) >= eps
    if mask.sum() > 0:
        safe_mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100.0
    else:
        safe_mape = np.nan

    # SMAPE: stable even near zero
    denom = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    denom = np.where(denom < 1e-6, 1e-6, denom)
    smape = np.mean(np.abs(y_true - y_pred) / denom) * 100.0

    return {
        "MAE": float(mae),
        "RMSE": float(rmse),
        "R2": float(r2),
        "SafeMAPE_percent_(|y|>=eps)": None if np.isnan(safe_mape) else float(safe_mape),
        "SMAPE_percent": float(smape),
    }
    
def within_tolerance_accuracy(y_true, y_pred, tol=150.0):
    y_true = np.asarray(y_true).reshape(-1)
    y_pred = np.asarray(y_pred).reshape(-1)
    return (np.mean(np.abs(y_true - y_pred) <= tol) * 100.0)

def plot_training(train_losses, val_losses, outpath):
    plt.figure()
    plt.plot(train_losses, label="train_loss")
    plt.plot(val_losses, label="val_loss")
    plt.xlabel("Epoch")
    plt.ylabel("MSE Loss")
    plt.legend()
    plt.tight_layout()
    plt.savefig(outpath)
    plt.close()

def plot_pred_vs_actual(days, y_true, y_pred, outpath):
    order = np.argsort(days)
    days = np.array(days)[order]
    y_true = np.array(y_true)[order]
    y_pred = np.array(y_pred)[order]

    plt.figure()
    plt.plot(days, y_true, label="Actual")
    plt.plot(days, y_pred, label="Predicted")
    plt.xlabel("Day")
    plt.ylabel("Total Qty")
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.savefig(outpath)
    plt.close()

def plot_residual_hist(residuals, outpath):
    plt.figure()
    plt.hist(residuals, bins=40)
    plt.xlabel("Deviation (Actual - Predicted)")
    plt.ylabel("Count")
    plt.tight_layout()
    plt.savefig(outpath)
    plt.close()

def plot_score_trend(days, scores, outpath):
    order = np.argsort(days)
    days = np.array(days)[order]
    scores = np.array(scores)[order]

    plt.figure()
    plt.plot(days, scores)
    plt.xlabel("Day")
    plt.ylabel("Irregularity Score (higher = more unusual)")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(outpath)
    plt.close()

In [48]:
#Step-4  :  Loading & Cleaning the columns of the dataset

def load_and_clean(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)

    # Your CSV columns confirmed:
    # ['Site','Type','Date','Number','Class','Site.1','Item','Qty','Amount','Balance']
    df["station_id"] = STATION_ID
    df["tank_id"] = df["Site.1"].astype(str).str.strip()           # stable tank identity per row
    df["date"] = pd.to_datetime(df["Date"], errors="coerce")
    df["fuel_type"] = df["Item"].apply(normalize_fuel)

    df["qty"] = pd.to_numeric(df["Qty"], errors="coerce")
    df["amount"] = df["Amount"].apply(to_float)
    df["balance"] = df["Balance"].apply(to_float)

    df = df.dropna(subset=["date", "tank_id", "fuel_type", "qty"]).copy()
    df = df[df["tank_id"].str.lower() != "nan"].copy()

    df = df.sort_values(["station_id", "tank_id", "fuel_type", "date"]).reset_index(drop=True)
    return df

In [49]:
#Step-5  :  Building Daily Behavior

def build_daily_behavior(df: pd.DataFrame) -> pd.DataFrame:
    daily = (
        df.groupby(["station_id", "tank_id", "fuel_type", df["date"].dt.date])
          .agg(
              total_qty=("qty", "sum"),
              txn_count=("qty", "count"),
              avg_txn=("qty", "mean"),
              std_txn=("qty", "std"),
              closing_balance=("balance", "last"),
          )
          .reset_index()
          .rename(columns={"date": "day"})
    )

    daily["day"] = pd.to_datetime(daily["day"])
    daily = daily.sort_values(["station_id", "tank_id", "fuel_type", "day"]).reset_index(drop=True)

    daily["std_txn"] = daily["std_txn"].fillna(0.0)
    daily["dow"] = daily["day"].dt.dayofweek
    daily["is_weekend"] = (daily["dow"] >= 5).astype(int)

    daily["balance_delta"] = daily.groupby(["station_id", "tank_id", "fuel_type"])["closing_balance"].diff().fillna(0.0)
    daily["qty_vs_balance_gap"] = (daily["total_qty"] - daily["balance_delta"].abs()).fillna(0.0)

    return daily

In [50]:
#Step-6  :  Build Sequences

FEATURES = [
    "txn_count", "avg_txn", "std_txn",
    "balance_delta", "qty_vs_balance_gap",
    "dow", "is_weekend"
]
TARGET = "qty_change"


def make_sequences(group_df: pd.DataFrame):
    X, y, meta = [], [], []
    vals = group_df[FEATURES].values.astype(np.float32)
    tgt = group_df[TARGET].values.astype(np.float32)
    days = group_df["day"].values

    for i in range(LOOKBACK, len(group_df)):
        X.append(vals[i - LOOKBACK:i])
        y.append(tgt[i])
        meta.append({
            "station_id": group_df["station_id"].iloc[i],
            "tank_id": group_df["tank_id"].iloc[i],
            "fuel_type": group_df["fuel_type"].iloc[i],
            "day": pd.to_datetime(days[i])
        })

    if not X:
        return None, None, None
    return np.array(X), np.array(y), pd.DataFrame(meta)

def build_all_sequences(daily: pd.DataFrame):
    Xs, ys, metas = [], [], []

    for _, g in daily.groupby(["station_id", "tank_id", "fuel_type"]):
        g = g.sort_values("day").reset_index(drop=True)
        if len(g) < LOOKBACK + 10:
            continue

        X, y, m = make_sequences(g)
        if X is None:
            continue

        Xs.append(X)
        ys.append(y)
        metas.append(m)

    if len(Xs) == 0:
        raise ValueError("Not enough data to build sequences. Add more months or reduce LOOKBACK.")

    X = np.concatenate(Xs, axis=0)
    y = np.concatenate(ys, axis=0)
    meta = pd.concat(metas, ignore_index=True).sort_values("day").reset_index(drop=True)
    return X, y, meta

In [51]:
#Step-7  :  PyTorch TCN Model

class FuelSeqDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)         # [N, T, F]
        self.y = torch.tensor(y, dtype=torch.float32).view(-1, 1)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

class Chomp1d(nn.Module):
    def __init__(self, chomp_size: int):
        super().__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size] if self.chomp_size > 0 else x

class TemporalBlock(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_size, dilation):
        super().__init__()
        padding = (kernel_size - 1) * dilation

        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size, padding=padding, dilation=dilation)
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()

        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size, padding=padding, dilation=dilation)
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()

        self.downsample = nn.Conv1d(in_ch, out_ch, 1) if in_ch != out_ch else None
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.relu1(self.chomp1(self.conv1(x)))
        out = self.relu2(self.chomp2(self.conv2(out)))
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)

class TCNRegressor(nn.Module):
    def __init__(self, num_inputs, channels=(16, 16, 16, 16), kernel_size=3):
        super().__init__()
        dilations = (1, 2, 4, 8)

        blocks = []
        in_ch = num_inputs
        for out_ch, d in zip(channels, dilations):
            blocks.append(TemporalBlock(in_ch, out_ch, kernel_size, dilation=d))
            in_ch = out_ch

        self.tcn = nn.Sequential(*blocks)
        self.head = nn.Sequential(
            nn.AdaptiveAvgPool1d(1),
            nn.Flatten(),
            nn.Linear(in_ch, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        x = x.transpose(1, 2)   # [N,T,F] -> [N,F,T]
        z = self.tcn(x)
        return self.head(z)

In [52]:
#Step-8  :  Training & Evaluating TCN model

def train_model(X_train, y_train, epochs, batch_size, lr):
    set_seeds(SEED)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = TCNRegressor(num_inputs=X_train.shape[-1]).to(device)

    ds = FuelSeqDataset(X_train, y_train)
    n = len(ds)
    n_val = int(n * 0.2)
    n_tr = n - n_val
    tr_subset, va_subset = torch.utils.data.random_split(
        ds, [n_tr, n_val], generator=torch.Generator().manual_seed(SEED)
    )

    tr_loader = DataLoader(tr_subset, batch_size=batch_size, shuffle=True)
    va_loader = DataLoader(va_subset, batch_size=batch_size, shuffle=False)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_val = float("inf")
    patience = 12
    pat = 0

    train_losses, val_losses = [], []

    for epoch in range(epochs):
        model.train()
        tr_loss = 0.0
        for xb, yb in tr_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()
            tr_loss += loss.item() * xb.size(0)
        tr_loss /= len(tr_subset)

        model.eval()
        va_loss = 0.0
        with torch.no_grad():
            for xb, yb in va_loader:
                xb, yb = xb.to(device), yb.to(device)
                pred = model(xb)
                loss = criterion(pred, yb)
                va_loss += loss.item() * xb.size(0)
        va_loss /= len(va_subset)

        train_losses.append(tr_loss)
        val_losses.append(va_loss)

        # Early stopping
        if va_loss < best_val - 1e-6:
            best_val = va_loss
            pat = 0
            torch.save(model.state_dict(), os.path.join(OUTDIR, "tcn_behavior_model.pt"))
        else:
            pat += 1
            if pat >= patience:
                break

    # load best
    model.load_state_dict(torch.load(os.path.join(OUTDIR, "tcn_behavior_model.pt"), map_location=device))
    plot_training(train_losses, val_losses, os.path.join(OUTDIR, "training_loss.png"))
    return model, device

def predict(model, device, X):
    loader = DataLoader(torch.tensor(X, dtype=torch.float32), batch_size=256, shuffle=False)
    preds = []
    model.eval()
    with torch.no_grad():
        for xb in loader:
            xb = xb.to(device)
            preds.append(model(xb).cpu().numpy().reshape(-1))
    return np.concatenate(preds)

In [53]:
#Step-9  :  Score Irregularities

def group_events(scored_df: pd.DataFrame, quantile=0.95) -> pd.DataFrame:
    d = scored_df.sort_values(["station_id", "tank_id", "fuel_type", "day"]).copy()
    thr = d["irregularity_score"].quantile(quantile)
    d["flag"] = (d["irregularity_score"] >= thr).astype(int)

    events = []
    for (st, tank, fuel), g in d.groupby(["station_id", "tank_id", "fuel_type"]):
        g = g.sort_values("day")
        active = None

        for _, row in g.iterrows():
            if row["flag"] == 1 and active is None:
                active = {
                    "station_id": st, "tank_id": tank, "fuel_type": fuel,
                    "start": row["day"], "end": row["day"],
                    "max_score": float(row["irregularity_score"]),
                    "max_abs_deviation": float(abs(row["deviation"]))
                }
            elif row["flag"] == 1 and active is not None:
                active["end"] = row["day"]
                active["max_score"] = float(max(active["max_score"], row["irregularity_score"]))
                active["max_abs_deviation"] = float(max(active["max_abs_deviation"], abs(row["deviation"])))
            elif row["flag"] == 0 and active is not None:
                events.append(active)
                active = None

        if active is not None:
            events.append(active)

    ev = pd.DataFrame(events)
    if len(ev) > 0:
        ev["threshold_used"] = thr
    return ev

In [57]:
#Step-10  :  Main Pipeline

def main():
    os.makedirs(OUTDIR, exist_ok=True)
    set_seeds(SEED)

    print("1) Load + clean")
    raw = load_and_clean(DATA_PATH)
    raw.to_csv(os.path.join(OUTDIR, "raw_cleaned.csv"), index=False)

    print("2) Build daily behavior dataset")
    daily = build_daily_behavior(raw)
    daily["qty_change"] = daily.groupby(["station_id","tank_id","fuel_type"])["total_qty"].diff().fillna(0.0)
    daily.to_csv(os.path.join(OUTDIR, "daily_behavior_dataset.csv"), index=False)

    print("3) Build sequences")
    X, y, meta = build_all_sequences(daily)

    print("4) Time split (train/test)")
    cutoff = meta["day"].quantile(TRAIN_QUANTILE)
    train_mask = meta["day"] <= cutoff
    test_mask = meta["day"] > cutoff

    X_train_raw, y_train = X[train_mask.values], y[train_mask.values]
    X_test_raw, y_test = X[test_mask.values], y[test_mask.values]
    meta_train = meta[train_mask].reset_index(drop=True)
    meta_test = meta[test_mask].reset_index(drop=True)

    print(f"Train samples: {len(y_train)} | Test samples: {len(y_test)}")

    print("5) Scale features (fit on train only)")
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train_raw.reshape(-1, X_train_raw.shape[-1])).reshape(X_train_raw.shape)
    X_test = scaler.transform(X_test_raw.reshape(-1, X_test_raw.shape[-1])).reshape(X_test_raw.shape)

    print("6) Train TCN model")
    model, device = train_model(X_train, y_train, EPOCHS, BATCH_SIZE, LR)

    print("7) Evaluate forecasting")
    y_pred_train = predict(model, device, X_train)
    y_pred_test = predict(model, device, X_test)

    train_metrics = compute_metrics(y_train, y_pred_train)
    test_metrics = compute_metrics(y_test, y_pred_test)

    print("\nModel Test Metrics:")
    for k, v in test_metrics.items():
        print(f"  {k}: {v}")
        
    acc_100 = within_tolerance_accuracy(y_test, y_pred_test, tol=100.0)
    acc_150 = within_tolerance_accuracy(y_test, y_pred_test, tol=150.0)
    acc_200 = within_tolerance_accuracy(y_test, y_pred_test, tol=200.0)

    print("\nAccuracy (within tolerance):")
    print(f"  ±100 L : {acc_100:.2f}%")
    print(f"  ±150 L : {acc_150:.2f}%")
    print(f"  ±200 L : {acc_200:.2f}%")

# =========================
# CORRECT BASELINE for qty_change
# =========================
    baseline = np.roll(y_test, 1)
    baseline[0] = 0.0

    baseline_metrics = compute_metrics(y_test, baseline)
    print("\nBaseline Test Metrics:")
    for k, v in baseline_metrics.items():
        print(f"  {k}: {v}")
    
    with open(os.path.join(OUTDIR, "forecast_metrics.json"), "w") as f:
        json.dump({"train": train_metrics, "test": test_metrics}, f, indent=2)

    # Charts
    plot_pred_vs_actual(meta_test["day"].values, y_test, y_pred_test,
                        os.path.join(OUTDIR, "pred_vs_actual_test.png"))
    residual_test = (y_test - y_pred_test)
    plot_residual_hist(residual_test, os.path.join(OUTDIR, "residuals_hist_test.png"))

    print("8) Score irregularities from residual patterns")
    residual_train = (y_train - y_pred_train)

    iso = IsolationForest(n_estimators=300, contamination="auto", random_state=SEED)
    iso.fit(np.vstack([residual_train, np.abs(residual_train)]).T)

    score_test = -iso.score_samples(np.vstack([residual_test, np.abs(residual_test)]).T)
    plot_score_trend(meta_test["day"].values, score_test,
                     os.path.join(OUTDIR, "irregularity_scores.png"))

    scored = meta_test.copy()
    scored["actual_total_qty"] = y_test
    scored["pred_total_qty"] = y_pred_test
    scored["deviation"] = residual_test
    scored["irregularity_score"] = score_test
    scored.to_csv(os.path.join(OUTDIR, "scored_days_test.csv"), index=False)

    print("9) Group consecutive flagged days into events")
    events = group_events(scored, SCORE_QUANTILE)
    events.to_csv(os.path.join(OUTDIR, "grouped_events.csv"), index=False)

    print("10) Save artifacts")
    joblib.dump(scaler, os.path.join(OUTDIR, "feature_scaler.pkl"))
    joblib.dump(iso, os.path.join(OUTDIR, "irregularity_scorer.pkl"))
    with open(os.path.join(OUTDIR, "model_config.json"), "w") as f:
        json.dump({"lookback": LOOKBACK, "features": FEATURES}, f, indent=2)

    # model already saved during training as tcn_behavior_model.pt
    print("\n✅ DONE. Outputs saved in:", OUTDIR)
    print("\nForecast Test Metrics:")
    for k, v in test_metrics.items():
        print(f"  {k}: {v}")

if __name__ == "__main__":
    main()

1) Load + clean
2) Build daily behavior dataset
3) Build sequences
4) Time split (train/test)
Train samples: 328 | Test samples: 82
5) Scale features (fit on train only)
6) Train TCN model
7) Evaluate forecasting

Model Test Metrics:
  MAE: 118.74420166015625
  RMSE: 157.90322268520995
  R2: -1.537799835205078e-05
  SafeMAPE_percent_(|y|>=eps): 99.99174499511719
  SMAPE_percent: 199.4119110107422

Accuracy (within tolerance):
  ±100 L : 57.32%
  ±150 L : 74.39%
  ±200 L : 81.71%

Baseline Test Metrics:
  MAE: 197.8634033203125
  RMSE: 259.22613197746864
  R2: -1.695145845413208
  SafeMAPE_percent_(|y|>=eps): 157.24966430664062
  SMAPE_percent: 165.41452026367188
8) Score irregularities from residual patterns
9) Group consecutive flagged days into events
10) Save artifacts

✅ DONE. Outputs saved in: ml_outputs

Forecast Test Metrics:
  MAE: 118.74420166015625
  RMSE: 157.90322268520995
  R2: -1.537799835205078e-05
  SafeMAPE_percent_(|y|>=eps): 99.99174499511719
  SMAPE_percent: 199.411